# GeoNergyAna Colab Launcher (Refined Gas‑Phase + ML)
This Colab notebook runs **enegeo_refined4_gas_ml.py** fully in‑memory, including ML regression and classification:
1. Install dependencies (ML extras)
2. Upload your `.sph`, receptor PDB, and ZIP of poses
3. Unzip & organize inputs
4. Pull & exec pipeline script from GitHub
5. Inspect CSV & plots (incl. SHAP outputs)
6. Zip & download results


In [ ]:
# 1️⃣ Install dependencies
!pip install numpy pandas matplotlib scipy mdtraj biopython openmm scikit-learn lightgbm shap joblib git+https://github.com/openmm/pdbfixer.git

In [ ]:
# 2️⃣ Upload inputs
from google.colab import files
print("Upload your surface `.sph` file:")
sph_up = files.upload()
print("Upload receptor PDB file:")
rec_up = files.upload()
print("Upload ZIP of your poses directory (.pdb files):")
poses_up = files.upload()

In [ ]:
# 3️⃣ Unzip & organize inputs
import os, zipfile
sph_fname = next(iter(sph_up))
rec_fname = next(iter(rec_up))
poses_zip = next(iter(poses_up))
os.makedirs('user_inputs/poses', exist_ok=True)
os.replace(sph_fname, f'user_inputs/{sph_fname}')
os.replace(rec_fname, f'user_inputs/{rec_fname}')
with zipfile.ZipFile(poses_zip, 'r') as z: z.extractall('user_inputs/poses')

In [ ]:
# 4️⃣ Pull & run pipeline
import requests, sys
args = [
  'enegeo_refined4_gas_ml.py',
  '--sph', f'user_inputs/{sph_fname}',
  '--rec', f'user_inputs/{rec_fname}',
  '--ligdir', 'user_inputs/poses/ligs',
  '--outdir', 'results',
  '--frac_thresh', '0.5', '--depth_thresh', '0.3', '--vicinity_radius', '2.0',
  '--keep_h', '--nsteps', '2000', '--dt', '0.001', '--interval', '100',
  '--ml', '--hpo', '--classify', '--verbose'
]
sys.argv = args
script = requests.get('https://raw.githubusercontent.com/sciencemaths-collab/geonergyana/main/enegeo_refined4_gas_ml.py').text
exec(script)

In [ ]:
# 5️⃣ Inspect results
import pandas as pd
from IPython.display import display, Image
print('Files in results/:')
!ls results
df = pd.read_csv('results/kept_ligs.csv'); display(df)
for img in ['A_frac_depth.png','C_score_vs_wscore.png','E_score_vs_dG.png']: display(Image(f'results/{img}'))

In [ ]:
# 6️⃣ Zip & download
from google.colab import files
!zip -r results.zip results
files.download('results.zip')